# Clean Questionnaire Data Screening

In [5]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps
import biopsykit as bp

from empkins_io.datasets.d03.micro_gapvii import MicroBaseDataset

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

In [3]:
deploy_type = "local"

In [4]:
config_dict = json.load(Path("../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

PosixPath('/Volumes/luca_ssd/Study_Data/2022_12_GAPVII_Micro')

In [7]:
dataset = MicroBaseDataset(base_path)
dataset

,subject,condition,phase
0,VP_001,tsst,Prep
1,VP_001,tsst,Pause_1
2,VP_001,tsst,Talk
3,VP_001,tsst,Pause_2
4,VP_001,tsst,Pause_3
...,...,...,...
1691,VP_110,ftsst,Pause_2
1692,VP_110,ftsst,Pause_3
1693,VP_110,ftsst,Math
1694,VP_110,ftsst,Pause_4


## Merge 1.0 and 1.1 for all questionnaires

In [33]:
for time in ["screening", "pre", "post"]:
    # load 1.0
    data1 = pd.read_csv(base_path.joinpath(f"data_tabular/questionnaire/cleaned/{time}_1_0_cleaned.csv"), sep=";")

    # load 1.1
    data11 = pd.read_csv(base_path.joinpath(f"data_tabular/questionnaire/cleaned/{time}_1_1_cleaned.csv"), sep=";")

    # append
    data = pd.concat((data1,data11))

    # save
    data.to_excel(base_path.joinpath(f"data_tabular/questionnaire/processed/unipark_{time}.xlsx"))

In [30]:
code_mapping = dataset.code_mapping
code_mapping = code_mapping.reset_index().set_index("Code")
code_mapping.head()

,subject
Code,
LI17P,VP_001
IH30R,VP_002
HB25M,VP_003
SK29B,VP_004
BK24I,VP_005


## Load Questionnaire Data and Map Code to VP-ID

In [35]:
quest_data = pd.read_excel(base_path.joinpath("data_tabular/questionnaires/processed/unipark_screening.xlsx"))
quest_data = quest_data.rename(columns={"VPN_Kennung": "Code"})
quest_data = quest_data.set_index("Code")

quest_data = quest_data.join(code_mapping)

quest_data = quest_data.dropna(subset=["subject"]).set_index("subject")

quest_data

,Unnamed: 0,lfdn,external_lfdn,tester,dispcode,lastpage,quality,duration,Einverstaendnis_1,Einverstaendnis_2,...,FFA_09,FFA_10,FFA_11,FFA_12,FFA_13,FFA_14,v_510,rts7364387,rts7364406,rts7374062
subject,,,,,,,,,,,,,,,,,,,,,
VP_074,73,1554,0,0,31,7176240,-77,533,1,1,...,-77.0,-77.0,-77.0,-77.0,-77,-77,-66,191.0,0.0,0.0
VP_074,207,1858,0,0,31,7176254,-77,1796,1,1,...,3.0,3.0,3.0,3.0,2,2,Erste Vorlesung biologische Psychologie,150.0,0.0,1794.0
VP_009,188,468,0,0,31,7176254,-77,1898,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VP_033,329,823,0,0,31,7176254,-77,9010,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VP_006,51,111,0,0,31,7176254,-77,1217,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VP_094,274,2012,0,0,31,7176254,-77,3761,1,1,...,4.0,4.0,3.0,3.0,2,4,Flyer,93.0,0.0,3756.0
VP_056,42,1447,0,0,32,7176254,-77,-1,1,1,...,1.0,3.0,3.0,2.0,2,3,Aushang in den Blockhelden,50320.0,0.0,53667.0
VP_014,106,225,0,0,31,7176254,-77,1831,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# keep last entry for each subject
quest_data = quest_data.groupby("subject").last()

In [37]:
quest_data.replace(-77, np.nan, inplace=True)

In [38]:
quest_data

,Unnamed: 0,lfdn,external_lfdn,tester,dispcode,lastpage,quality,duration,Einverstaendnis_1,Einverstaendnis_2,...,FFA_09,FFA_10,FFA_11,FFA_12,FFA_13,FFA_14,v_510,rts7364387,rts7364406,rts7374062
subject,,,,,,,,,,,,,,,,,,,,,
VP_001,28,57,0,0,31,7176254,NaN,2793,1,1,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN
VP_002,60,126,0,0,32,7176254,NaN,-1,1,1,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN
VP_003,24,51,0,0,32,7176254,NaN,-1,1,1,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN
VP_004,48,104,0,0,32,7176254,NaN,-1,1,1,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN
VP_005,78,165,0,0,32,7176254,NaN,-1,1,1,...,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VP_099,212,1869,0,0,31,7176254,NaN,2271,1,1,...,2.0,1.0,1.0,1.0,2,2,"Freunde, die die Flyeraktion mitbekommen haben...",150.0,0.0,2268.0
VP_100,268,1998,0,0,32,7176254,NaN,-1,1,1,...,3.0,3.0,2.0,2.0,3,2,"Durch ein Plakat, was bei der Südmensa hing am...",76926.0,0.0,79774.0
VP_101,296,2067,0,0,31,7176254,NaN,2686,1,1,...,2.0,1.0,2.0,2.0,1,3,-99,245.0,0.0,2679.0


In [203]:
# load renaming json
renaming_dict = json.load(Path("renaming.json").open(encoding="utf-8"))

renaming_dict

{'Geschlecht': 'Gender',
 'Alter': 'Age',
 'Geburtsdatum': 'Birth_Date',
 'Familienstand': 'Marital',
 'Bildungsabschluss': 'Education',
 'Beschaeftigungsverhaeltnis': 'Profession',
 'Beschaeftigungsverhaeltnis_Sonstige': 'Profession_Other',
 'Ethnische_Herkunft': 'Ethnicity',
 'Studiengang': 'Course_Study',
 'Berufsfeld': 'Professional_Field',
 'Muttersprache': 'Native_Language',
 'Koerpergroesse': 'Height',
 'Koerpergewicht': 'Weight',
 'Haendigkeit': 'Handedness',
 'Kontrazeptiva_Keine': 'Contraceptives_None',
 'Kontrazeptiva_Pille': 'Contraceptives_Pill',
 'Kontrazeptiva_hormonfrei': 'Contraceptives_Hormone_Free',
 'Kontrazeptiva_hormonell_andere': 'Contraceptives_Hormone_Other',
 'Menstruation_regelm_Zyklus': 'Menstruation_Regular_Cycle',
 'Menstruation_unregelm_Zyklus_Grund': 'Menstruation_Unregular_Cycle_Reason',
 'Menstruation_letzte_Regelblutung': 'Menstruation_Last_Period',
 'Zykluslaenge': 'Cycle_Length',
 'Menstruation_Zyklus_Schwankung': 'Menstruation_Cycle_Fluctuations',


In [204]:
quest_data = quest_data.rename(renaming_dict, axis=1)

# get only columns that  are in renaming dict and in the quest_data
columns = list(set(list(renaming_dict.values())) & set(quest_data.columns))

quest_data_filtered = quest_data[columns]

quest_data_filtered.head()

,Disease_Skeleton,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,Menstruation_Cycle_Fluctuations,Date_Ovulation,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,2,2,1,2,5,2022-01-06,0.0,4.0,2022-02-16,...,2,2,2,-99,2,2,2,2.0,2,28
VP_02,2,1,2,1,2,6,2022-03-06,NaN,1.0,-66,...,2,2,2,-99,2,2,2,1.0,2,29
VP_03,2,1,2,1,2,6,-66,NaN,NaN,-66,...,2,2,2,-99,2,2,2,NaN,2,23
VP_04,2,1,2,1,2,5,-66,NaN,NaN,-66,...,2,2,2,-99,2,2,2,NaN,2,24
VP_05,2,1,2,1,2,5,-66,NaN,NaN,-66,...,2,2,2,-99,2,2,2,NaN,2,23


In [205]:
# replace all missing values
quest_data_filtered = quest_data_filtered.replace("-66", np.nan)
quest_data_filtered = quest_data_filtered.replace(-77, np.nan)

In [206]:
quest_data_filtered

,Disease_Skeleton,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,Menstruation_Cycle_Fluctuations,Date_Ovulation,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,2,2,1,2,5,2022-01-06,0.0,4.0,2022-02-16,...,2,2,2,-99,2,2,2,2.0,2,28
VP_02,2,1,2,1,2,6,2022-03-06,NaN,1.0,NaN,...,2,2,2,-99,2,2,2,1.0,2,29
VP_03,2,1,2,1,2,6,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_04,2,1,2,1,2,5,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,24
VP_05,2,1,2,1,2,5,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_06,2,1,2,1,2,5,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,19
VP_07,2,2,2,1,2,4,2022-03-14,1.0,4.0,2022-03-23,...,2,2,2,-99,2,2,2,2.0,2,23
VP_08,2,1,2,1,2,5,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,22
VP_09,2,1,2,1,2,5,2022-03-01,NaN,2.0,NaN,...,2,2,2,-99,2,2,2,1.0,2,25


## Add Condition Order

In [207]:
condition_order = pd.read_csv(base_path.joinpath("data_tabular/extras/condition_order.csv"))
condition_order = condition_order.set_index("subject")["condition_order"]
condition_order.head()

# change condtion order to codes
condition_order = condition_order.replace("tsst_first", 1)
condition_order = condition_order.replace("ftsst_first", 2)

quest_data_filtered.insert(1, "Condition_Order", condition_order)

In [208]:
quest_data_filtered

,Disease_Skeleton,Condition_Order,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,Menstruation_Cycle_Fluctuations,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,1,2,2,1,2,5,2022-01-06,0.0,4.0,...,2,2,2,-99,2,2,2,2.0,2,28
VP_02,2,1,1,2,1,2,6,2022-03-06,NaN,1.0,...,2,2,2,-99,2,2,2,1.0,2,29
VP_03,2,2,1,2,1,2,6,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_04,2,2,1,2,1,2,5,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,24
VP_05,2,2,1,2,1,2,5,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_06,2,2,1,2,1,2,5,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,19
VP_07,2,1,2,2,1,2,4,2022-03-14,1.0,4.0,...,2,2,2,-99,2,2,2,2.0,2,23
VP_08,2,1,1,2,1,2,5,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,22
VP_09,2,1,1,2,1,2,5,2022-03-01,NaN,2.0,...,2,2,2,-99,2,2,2,1.0,2,25


In [209]:
# add to codebook

codebook = pd.read_csv(base_path.joinpath("data_tabular/questionnaires/codebook.csv"))
codebook.set_index("variable", inplace=True)

In [210]:
codebook

,1,2,3,4,5,6,7,8,9,10,11,12,13
variable,,,,,,,,,,,,,
Gender,female,male,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education,kein Schulabschluss,Hauptschule,Mittlere Reife,Fachabitur,Abitur,Bachelor,Master/Diplom,Promotion,Habilitation,NaN,NaN,NaN,NaN
Marital,ledig,feste Partnerschaft,verheiratet,getrennt lebend,geschieden,verwitwet,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Profession,Arbeitslos/Arbeitssuchend,Auszubildende/r,Student/in,Hausfrau/mann,Angestellte/r,Beamte/in,Selbstständig,Elternzeit,Beurlaubt,Sabbatjahr,Berufsunfähig/Frührente,Ruhestand,sonstiges
Ethnicity,Asiatisch,Arabisch,Afrikanisch,Indigen,Weiß,Andere,Keine Antwort,NaN,NaN,NaN,NaN,NaN,NaN
Condition_Order,tsst_first,ftsst_first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Handedness,right,left,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [211]:
# add row for condition order with nan values
codebook.loc["Condition_Order"] = np.nan

codebook.loc["Condition_Order", "1"] = "tsst_first"
codebook.loc["Condition_Order", "2"] = "ftsst_first"

In [212]:
# export
codebook.to_csv(base_path.joinpath("data_tabular/questionnaires/codebook.csv"))

## Add Handedness

In [213]:
handedness = pd.read_excel(base_path.joinpath("data_tabular/extras/handedness.xlsx"))
handedness = handedness.set_index("subject")["handedness"]

# recode handedness
handedness = handedness.replace("right", 1)
handedness = handedness.replace("left", 2)

quest_data_filtered.insert(1, "Handedness", handedness)
quest_data_filtered.head()

,Disease_Skeleton,Handedness,Condition_Order,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,1,1,2,2,1,2,5,2022-01-06,0.0,...,2,2,2,-99,2,2,2,2.0,2,28
VP_02,2,1,1,1,2,1,2,6,2022-03-06,NaN,...,2,2,2,-99,2,2,2,1.0,2,29
VP_03,2,1,2,1,2,1,2,6,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_04,2,2,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,24
VP_05,2,1,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23


In [214]:
# add to codebook

# add row for handedness with nan values
codebook.loc["Handedness"] = np.nan

codebook.loc["Handedness", "1"] = "right"
codebook.loc["Handedness", "2"] = "left"

# export
codebook.to_csv(base_path.joinpath("data_tabular/questionnaires/codebook.csv"))

In [215]:
quest_data_filtered

,Disease_Skeleton,Handedness,Condition_Order,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,1,1,2,2,1,2,5,2022-01-06,0.0,...,2,2,2,-99,2,2,2,2.0,2,28
VP_02,2,1,1,1,2,1,2,6,2022-03-06,NaN,...,2,2,2,-99,2,2,2,1.0,2,29
VP_03,2,1,2,1,2,1,2,6,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_04,2,2,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,24
VP_05,2,1,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_06,2,1,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,19
VP_07,2,1,1,2,2,1,2,4,2022-03-14,1.0,...,2,2,2,-99,2,2,2,2.0,2,23
VP_08,2,1,1,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,22
VP_09,2,1,1,1,2,1,2,5,2022-03-01,NaN,...,2,2,2,-99,2,2,2,1.0,2,25


## Export Base

In [216]:
quest_data_filtered.sort_index(axis=1, inplace=True)
quest_data_filtered.to_csv(base_path.joinpath("data_tabular/questionnaires/final/base_data.csv"))

## PASA

In [217]:
pasa_data = pd.read_excel(base_path.joinpath("data_tabular/questionnaires/raw/pasa.xlsx"))
pasa_data = pasa_data.set_index(["subject", "condition"])

# add "PASA_" to column names 
pasa_data.columns = "PASA_" + pasa_data.columns.astype(str).str.zfill(2)
pasa_data = pasa_data.unstack("condition")

pasa_data = pasa_data.sort_index(axis=1, level="condition")

# flatten multiindex
pasa_data.columns = ["_".join(col).strip() for col in pasa_data.columns.values]

pasa_data.head()


,PASA_01_ftsst,PASA_02_ftsst,PASA_03_ftsst,PASA_04_ftsst,PASA_05_ftsst,PASA_06_ftsst,PASA_07_ftsst,PASA_08_ftsst,PASA_09_ftsst,PASA_10_ftsst,...,PASA_07_tsst,PASA_08_tsst,PASA_09_tsst,PASA_10_tsst,PASA_11_tsst,PASA_12_tsst,PASA_13_tsst,PASA_14_tsst,PASA_15_tsst,PASA_16_tsst
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,4,5,5,5,2,2,0,4.0,2,3,...,3,4.0,0,1,2,3,4.0,4.0,1.0,4.0
VP_02,4,4,2,3,1,2,3,2.0,3,3,...,4,1.0,2,1,0,3,4.0,4.0,2.0,3.0
VP_03,5,3,4,3,1,2,1,3.0,5,3,...,0,4.0,4,2,3,3,2.0,3.0,3.0,4.0
VP_04,4,3,4,1,0,4,0,4.0,5,5,...,0,3.0,3,2,4,1,1.0,4.0,3.0,3.0
VP_05,2,2,1,3,4,1,3,3.0,2,2,...,3,3.0,2,2,2,2,3.0,3.0,3.0,2.0


In [218]:
pasa_data.to_csv(base_path.joinpath("data_tabular/questionnaires/final/pasa.csv"))

## ADS-L

In [219]:
quest_data.filter(like="ADS").to_csv(base_path.joinpath("data_tabular/questionnaires/final/ads.csv"))

## STADI

In [220]:
quest_data.filter(like="STADI").to_csv(base_path.joinpath("data_tabular/questionnaires/final/stadi.csv"))

## Brief Cope

In [221]:
quest_data.filter(like="Brief").to_csv(base_path.joinpath("data_tabular/questionnaires/final/brief_cope.csv"))

## PSS

In [222]:
quest_data.filter(like="PSS").to_csv(base_path.joinpath("data_tabular/questionnaires/final/pss.csv"))

## BFIK

In [224]:
quest_data.filter(like="BFIK").to_csv(base_path.joinpath("data_tabular/questionnaires/final/bfik.csv"))

## RSE

In [226]:
quest_data.filter(like="RSE").to_csv(base_path.joinpath("data_tabular/questionnaires/final/rse.csv"))

## SCS

In [228]:
quest_data.filter(like="SCS").to_csv(base_path.joinpath("data_tabular/questionnaires/final/scs.csv"))

## RSQ

In [230]:
quest_data.filter(like="RSQ").to_csv(base_path.joinpath("data_tabular/questionnaires/final/rsq.csv"))

## BES

In [232]:
quest_data.filter(like="BES").to_csv(base_path.joinpath("data_tabular/questionnaires/final/bes.csv"))

## SOC

In [233]:
quest_data.filter(like="SOC").to_csv(base_path.joinpath("data_tabular/questionnaires/final/soc.csv"))

## TSGS

In [235]:
quest_data.filter(like="TSGS").to_csv(base_path.joinpath("data_tabular/questionnaires/final/tsgs.csv"))